In [ ]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib as plt
%matplotlib inline

In [ ]:
import requests
# Link created from query, but modifying 'html' to 'json'
r = requests.get('http://services7.arcgis.com/yUlpDmY4xMAdbBIp/arcgis/rest/services/ParksRocks/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnDistinctValues=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&quantizationParameters=&sqlFormat=none&f=json&token=')

dfjson = r.json()
dfjson.keys()
features = dfjson['features']

JSON FORMAT

{u'attributes': {u'CreationDate': 1492223813720,
   u'Creator': u'srn0629_dev',
   u'Date': 1492128000000,
   u'EditDate': 1492223813720,
   u'Editor': u'srn0629_dev',
   u'GlobalID': u'f83571f6-2c9e-4d32-843a-2983b7f842ba',
   u'HidorFound': u'Hid',
   u'OBJECTID': 1,
   u'ParkName': u'Alderwood',
   u'Quantity': 3,
   u'RockersName': u'Savanna'},
  u'geometry': {u'x': 1145976.5501281046, u'y': 686346.8414406658}}


In [ ]:
for dct in features:
    for key, val in dct['attributes'].iteritems():
        dct[key] = val
    for key, val in dct['geometry'].iteritems():
        dct[key] = val
    del dct['attributes']
    del dct['geometry']

In [ ]:
features[0:2]

In [ ]:
#from datetime import datetime
df = pd.DataFrame(features)
df.head(1)
#newDF['CreationDate'] = pd.to_datetime(newDF['CreationDate'])#,infer_datetime_format = True)
#df['date'] = pd.to_datetime(df['date'],unit='s')
#newDF['Date'] = pd.to_datetime(newDF['Date'])
#newDF.head(1)

In [ ]:
# Number of events per park.
df['Date'].groupby(by = df['ParkName']).count()

In [ ]:
# Number of total rocks per park.
df['Quantity'].groupby(by = df['ParkName']).sum()

In [ ]:
# Number of hidden rocks per park.
totalrocksbypark = df[df['HidorFound']=='Hid']['Quantity'].groupby(by = df['ParkName']).sum()
totalrocksbypark

In [ ]:
parkplot = totalrocksbypark.plot.bar()

In [ ]:
fig = parkplot.get_figure()
fig.tight_layout()
fig.savefig("parkplot.jpg")

In [ ]:
df.head()

In [ ]:
# Number of hidden rock events per park.
parks_hid = pd.DataFrame(df[df['HidorFound']=='Hid']['Date'].groupby(by = df['ParkName']).count())
parks_hid.columns = ['Hide Events']
#df_year.columns = [str(year)]
parks_hid.head()

In [ ]:
# Number of hidden rocks per park.
parks_hidtotal = pd.DataFrame(df[df['HidorFound']=='Hid']['Date'].groupby(by = df['ParkName']).sum())
parks_hidtotal.columns = ['Hidden Rocks']
#df_year.columns = [str(year)]
parks_hidtotal.head()

In [ ]:
# Number of found rock events per park.
parks_found = pd.DataFrame(df[df['HidorFound']=='Found']['Date'].groupby(by = df['ParkName']).count())
parks_found.columns = ['Found Events']
#df_year.columns = [str(year)]
parks_found.head()

In [ ]:
#result = pd.concat([parks_hid, parks_found])
#result

parks_hid = parks_hid.join(parks_found, how = 'outer')
result = parks_hid.fillna(value = 0)
result

In [ ]:
stacked_bar = result.ix[:,result.columns].plot.bar(stacked = True, figsize = (16,6), fontsize = 20, legend = True);
stacked_bar.legend(fontsize = 20)
stacked_bar.set_xlabel("Park Name",fontsize = 20)

stacked_bar

In [ ]:
fig = stacked_bar.get_figure()
fig.tight_layout()
fig.savefig("stacked_bar.jpg")

In [ ]:
# Number of rocks each rocker has hid/found
df.groupby(by = df['RockersName'])['Quantity'].count()